In [1]:
import folium
import json
import numpy as np
import pandas as pd
from shapely.geometry import shape, Point

In [2]:
# generate Reindeer positions
nvalues = 10
lons = 7.48 + np.random.rand(nvalues,1)
lons

lats = 60.58 + np.random.rand(nvalues,1)
pd.DataFrame(np.concatenate((lons,lats),axis=1),  columns=["lon", "lat"]).to_csv("reindeers.csv", index=False)

In [3]:
zone1 = 'nordfjella.json'
geo_json_data = json.load(open(zone1))

In [13]:
lon, lat = 7.48, 60.58
zoom_start = 6

m = folium.Map(location=[lat, lon], tiles="Stamen Terrain",
                  zoom_start=zoom_start)
def color_zone1(feature):
    return {'opacity': 1, 
            'weight': 1, 
            'color': 'blue'}

zone1 = folium.FeatureGroup(name='Zone 1')
zone1.add_to(m)

folium.GeoJson(geo_json_data, style_function=color_zone1).add_to(zone1)


In [14]:
# Read reindeer positions
reindeers = pd.read_csv("reindeers.csv")
reindeers

,lon,lat
0,8.200042,61.115087
1,7.586293,61.351048
2,8.142887,60.923468
3,7.691862,61.087075
4,7.578464,61.290441
5,8.477666,61.546647
6,7.651247,61.509017
7,8.021040,60.945702
8,7.891361,60.991005
9,8.326906,61.201764


In [15]:
# Black if healthy and red if CWD
healthy=0
CWD=0
lon_in = 0.
lat_in = 0.
lon_out = 0.
lat_out = 0.
for lon,lat in zip(reindeers['lon'],reindeers['lat']):
    point = Point(lon,lat)
    polygon = shape(geo_json_data['geometry'])
    if point.within(polygon):
        print("INSIDE")
        lon_in = lon_in + lon
        lat_in = lat_in + lat
        healthy = healthy + 1
    else:
        print("OUTSIDE")        
        lon_out = lon_out + lon
        lat_out = lat_out + lat
        CWD = CWD + 1

if (healthy > 0):
    lon_in = lon_in / float(healthy)
    lat_in = lat_in / float(healthy)

if (CWD > 0):
    lon_out = lon_out / float(CWD)
    lat_out = lat_out / float(CWD)

OUTSIDE
OUTSIDE
INSIDE
OUTSIDE
OUTSIDE
OUTSIDE
OUTSIDE
INSIDE
INSIDE
OUTSIDE


In [16]:
# Read Reindeer healthy

reindeer_OK = folium.FeatureGroup(name='Healthy reindeer')

reindeer_OK.add_to(m)

icon_OK = folium.features.CustomIcon("./deer_OK.png",icon_size=(25, 25))

folium.Marker(
            [lat_out,lon_out],
            popup='Healthy Reindeer '+ str(healthy),
            icon = icon_OK
).add_to(reindeer_OK)

In [17]:
# Sick reindeer
# Reindeer healthy

reindeer_KO = folium.FeatureGroup(name='Reindeer with CWD')

reindeer_KO.add_to(m)

icon_KO = folium.features.CustomIcon("./deer_KO.png",icon_size=(25, 25))

folium.Marker(
            [lat_in,lon_in],
            popup='CWD Reindeer '+ str(CWD),
            icon = icon_KO
).add_to(reindeer_KO)

In [18]:
folium.LayerControl().add_to(m)


In [19]:
m